In [2]:
from jobseeker_agent.utils.paths import load_raw_job, load_prompt, load_review, get_data_path
from jobseeker_agent.scraper.extract_job_details import extract_job_details

job_id = 18
job = load_raw_job(job_id)
job_url = job["job_link"]
job_details = extract_job_details(job_url)
description = job_details["description"]
# description

# load profile
profile = load_prompt("profil_pro")

# synthesis and decision
review = load_review(job_id)
synthesis = review["synthesis_and_decision"]

resume_path = get_data_path() / "resume" / "18" / "resume.tex"
with open(resume_path, "r") as f:
    resume = f.read()

# cover letter template
cover_letter_path = get_data_path() / "resume" / "template" / "cover-letter-en.tex"
with open(cover_letter_path, "r") as f:
    cover_letter_template = f.read()

In [92]:
# Les prompts
system_prompt = r"""
You are a cover letter expert.
# Input data
## Job description
{job_description}
## Candidate profile
{profil_pro}
## Synthesis and decision
{synthesis_and_decision}
## Candidate resume
{resume}
"""

first_draft_prompt = r"""
## Cover letter template
{cover_letter_template}

# Task
Write a cover letter for the job by modifying the LaTeX template provided.
You MUST return the COMPLETE LaTeX file with all its structure (\documentclass, \usepackage, \begin{{document}}, etc.).

Instructions:
1. Replace the \companyname and \jobtitle commands with the actual company name and job title from the job description.
3. Modify ONLY the content between the comments:
   - For the first part (between "% first part starts here" and "% first part ends here"): Identify the company's mission and how it corresponds to the candidate's profile.
   - For the second part (between "% second part starts here" and "% second part ends here"): Highlight 2 or 3 **experiences or key skills** (hard and soft skills) that correspond to the requirements of the offer. **Illustrate** always with concrete examples or quantified results. Mention them in the same order as in the resume.
   - For the third part (between "% third part starts here" and "% third part ends here"): Write a wrap-up that summarizes what has been said by putting a term on my profile (e.g., "versatile problem solver"), and that reminds that I am fully aligned with their mission.
4. Keep everything else from the template unchanged (opening, closing, signature block, etc.).

Do not lie about the candidate's skills and expertise.

The cover letter should ideally be between 250 and 35à words long.
Do not mention languages spoken by the candidate.

Instead of general affirmations, explicitly link the candidate's specific past achievements, skills, or projects (mentioned later in the prompt) to the specific requirements or mission of the role and the company. The connection must be shown, not merely stated.
Bad examples: 
- "My experience directly addresses the core responsibilities of this role."
- "My experience is a strong fit for the Machine Learning Engineer role."

Return as plain text, no LaTeX code.
"""


# ## First draft of the cover letter
# {first_draft}
critic_prompt = r"""
# Task
Critic the cover letter.
Return a list of suggestions to improve the cover letter.
Return the suggestions in the following format:
- [suggestion 1]
- [suggestion 2]
- [suggestion 3]

Particular areas of interest:
- did the first draft lie about the candidate's skills and expertise ?
- is all information from the draft relevant to the job offer ?
- are experiences mentioned in decreasing order of relevance to the job offer ?
- In the first paragraph, is the link between the company's mission and the candidate's profile clear ?
"""

# ## Cover letter template
# {cover_letter_template}
# ## First draft of the cover letter
# {first_draft}
# ## Critic
# {critic}
corrector_prompt = r"""
# Task
Correct the cover letter based on the critic.
Return the corrected cover letter.
Return as plain text, no LaTeX code.
Previous drafw was {wordcount} words long.
"""

# ## Cover letter template
# {cover_letter_template}
# ## First draft of the cover letter
# {first_draft}
# ## Critic
# {critic}
# ## Second draft
# {cover_letter}
compressor_prompt = r"""
# Task
{wordcount_sentence}
Adjust the length of the cover letter to be between 250 and 350 words.
Return the compressed cover letter.
You may shorten in particular the first paragraph if it is too long.
Or simply remove an experience that is not that relevant for the job offer.
Forget about the LaTeX Formatting. It should be easy to read in a terminal. Output the content of the cover letter only, no LaTeX code.
"""


In [93]:
def write_cover_letter(job_description: str, profil_pro: str, synthesis_and_decision: str, resume: str, cover_letter_template: str, model: str = "gpt-5", status_callback=None) -> str:
    """Generate the first draft of the cover letter."""
    print("    [STAGE 1] Generating draft...")
    llm = get_llm(model)
    system_message = SystemMessage(content=system_prompt.format(
        job_description=job_description,
        profil_pro=profil_pro,
        synthesis_and_decision=synthesis_and_decision,
        resume=resume
    ))
    message = HumanMessage(content=first_draft_prompt.format(
        cover_letter_template=cover_letter_template
    )) 
    messages = [system_message, message]
    first_draft = llm.invoke(messages)
    messages.append(first_draft)
    print(f"    [STAGE 2] Critic...")
    if status_callback:
        status_callback("Stage 2/3: Critic...")
    message = HumanMessage(content=critic_prompt)
    messages.append(message)
    response = llm.invoke(messages)
    print(response.content)
    messages.append(response)
    print(f"    [STAGE 3] Corrector...")
    if status_callback:
        status_callback("Stage 3/3: Corrector...")
    message = HumanMessage(content=corrector_prompt.format(
        wordcount=len(first_draft.content.split())
    ))
    messages.append(message)
    revised_draft = llm.invoke(messages)
    messages.append(revised_draft)
    print(len(revised_draft.content.split()))
    if not(250 <= len(revised_draft.content.split()) <= 350):
        wordcount = len(revised_draft.content.split())
        if wordcount > 350:
            sentence = f"The letter should be between 300 and 400 words long. It is currently {wordcount} words long. It should be {int((wordcount - 300)/wordcount*100)}% shorter."
        else:
            sentence = f"The letter should be between 300 and 400 words long. It is currently {wordcount} words long. It should be {int((300 - wordcount)/wordcount*100)}% longer."
        print(f"    [STAGE 4] Compressor...")
        if status_callback:
            status_callback("Stage 4/4: Compressor...")
        
        message = HumanMessage(content=compressor_prompt.format(
            wordcount_sentence=sentence
        ))
        messages.append(message)
        revised_draft = llm.invoke(messages)
        messages.append(revised_draft)
    
    print(f"last draft wordcount: {len(revised_draft.content.split())}")
    return revised_draft.content

In [89]:
model = "gemini-2.5-flash"
result = write_cover_letter(description, profile, synthesis, resume, cover_letter_template, model)
print(result)

    [STAGE 1] Generating draft...
✅ Chargement du modèle Gemini : gemini-2.5-flash


E0000 00:00:1761567081.650205 7626704 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


    [STAGE 2] Critic...
Here are some suggestions to improve the cover letter:

-   **Strengthen the "production experience" aspect:** While the letter mentions "design to implementation," the job explicitly asks for "hands-on production experience." For the IBM project, if the PoC involved any deployment or integration steps, even if not full-scale production, it would be beneficial to highlight that. For the PhD and personal projects, emphasize the *readiness* and *capability* to deploy and maintain in production, given the end-to-end ownership.
-   **Acknowledge "Modern Data Stack" interest/knowledge:** The job description mentions "general knowledge of the 'modern data stack' ecosystem, especially data warehouses and databases." The candidate's profile notes an eagerness to learn this. A brief sentence expressing enthusiasm to learn or apply existing knowledge of data warehouses (even if theoretical) would be valuable.
-   **Refine the phrasing about AI generation:** The sentence "

In [71]:
not(250 <= len(result.split()) <= 350)

True

In [94]:
model = "gemini-2.5-pro"
result = write_cover_letter(description, profile, synthesis, resume, cover_letter_template, model)

    [STAGE 1] Generating draft...
✅ Chargement du modèle Gemini : gemini-2.5-pro


E0000 00:00:1761567702.738952 7626704 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


    [STAGE 2] Critic...
Here is a critique of the cover letter with suggestions for improvement:

- **Reorder the experiences in the second paragraph for maximum impact.** The job description's primary focus for the monitoring team is "time series forecasting models." Your experience at IBM is the most direct and powerful match for this core responsibility. Starting with the IBM project would immediately establish your relevance, followed by the agentic workflow project (which aligns with their newer generative AI goals), and finally the Ph.D. to demonstrate ownership and rigor.

- **Strengthen the connection in the first paragraph.** The current link is good but slightly generic. You can make it more specific by connecting your background in "autonomous decision-making systems" to their mission of "data observability." For example, you could state that your experience in building systems that *act* on complex data gives you a unique perspective on how to build systems that effectively

In [95]:
result

"Dear Sifflet Hiring Team,\n\nI am writing to express my enthusiastic interest in the Machine Learning Engineer position.\n\nSifflet's mission to build an intelligent data observability platform resonates perfectly with my background in designing autonomous decision-making systems. My experience building systems that act on complex data has prepared me to create the intelligent monitoring and anomaly detection solutions needed to end data quality troubleshooting.\n\nMy ability to contribute is demonstrated by my hands-on experience with your required technologies. At IBM, I developed a predictive maintenance tool using time series forecasting and anomaly detection to predict equipment failures—a skill set central to Sifflet's core monitoring capabilities for understanding seasonality and business cycles. More recently, I have been building an agentic workflow using modern frameworks like LangChain, which directly prepares me to develop the generative AI features mentioned in your roadm

In [73]:
model = "gpt-4o"
result = write_cover_letter(description, profile, synthesis, resume, cover_letter_template, model)
print(result)

    [STAGE 1] Generating draft...
✅ Chargement du modèle OpenAI : gpt-4o
    [STAGE 2] Critic...
- The first draft does not lie about the candidate's skills and expertise. It accurately reflects the candidate's background and aligns it with the job requirements.

- The information in the draft is mostly relevant to the job offer. However, the mention of the camera calibration project, while showcasing technical skills, could be more directly tied to the specific requirements of the job, such as time series forecasting or data observability.

- The experiences are not mentioned in decreasing order of relevance. The Ph.D. research at Thales, which directly relates to optimization and system automation, should be highlighted first as it aligns closely with the job's focus on deploying time series forecasting models and creating intelligent alerting systems. The camera calibration project, while impressive, is less directly related to the core responsibilities of the role and could be ment